In [1]:
sc = SparkContext.getOrCreate()
from pyspark.sql import SparkSession, Row, SQLContext 
spark = SparkSession.builder \
    .appName("Statistiques sur un texte avec Spark SQL") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
sqlContext = SQLContext(sc)

In [39]:
def text_load_and_processing(text_path):
    vocabulary = sc.textFile(text_path)\
        .flatMap(lambda lines: lines.lower().split())\
        .flatMap(lambda word: word.split(","))\
        .flatMap(lambda word: word.split("."))\
        .flatMap(lambda word: word.split(";"))\
        .flatMap(lambda word: word.split("!"))\
        .flatMap(lambda word: word.split("?"))\
        .flatMap(lambda word: word.split("\""))\
        .filter(lambda word: word is not None and len(word) > 0)\
        .filter(lambda word: word.find('@') == -1)\
        .filter(lambda word: word.find('/') == -1)\
        .filter(lambda word: word[0] != '-')\
        .filter(lambda word: word[len(word)-1] != '-')
    return vocabulary

In [3]:
def word_counts_fonction(lines):
    return lines.flatMap(lambda line: line.split(' '))\
        .map(lambda word: (word, 1))\
        .reduceByKey(lambda count1, count2: count1 + count2)

In [4]:
def Rdd_to_DataFrame(lines):
    header = Row('word', 'frequence')
    rdd = lines.map(lambda element: header(*element))
    return spark.createDataFrame(rdd)

def build_temporary_table(dataFrame):
    dataFrame.registerTempTable('wordCountTable')

In [47]:
lines = text_load_and_processing("iliad.mb.txt")
line = word_counts_fonction(lines)
df = Rdd_to_DataFrame(line)
build_temporary_table(df)

# le mot le plus long du texte

In [48]:
result =  sqlContext.sql('SELECT * FROM wordCountTable ORDER BY length(word) DESC LIMIT 20')
first_word = result.collect()[0].word
max_length = len(first_word)
print(' le(s) mot(s) le(s) long du text:')
for element in result.collect():
    if max_length == len(element.word):
        print " "
        print " {}".format(element.word)
    else:
        break

 le(s) mot(s) le(s) long du text 
 
 brothers-in-law-for


# le mot de quatre lettres le plus fréquent

In [49]:
result =  sqlContext.sql('SELECT * FROM wordCountTable where length(word) = 4 ORDER BY frequence DESC LIMIT 20')
if len(result.collect()) == 0:
    print "le text ne contient pas de mot de quatre lettre "
else:
    max_frequence = result.collect()[0].frequence
    print(' le(s) mot(s) de quatre lettres le(s) plus fréquent')
    for element in result.collect():
        if max_frequence == element.frequence:
            print " "
            print element.word
        else:
            break

 le(s) mot(s) de quatre lettres le(s) plus fréquent
 
with


# le mot de quinze lettres le plus fréquent

In [50]:
result =  sqlContext.sql('SELECT * FROM wordCountTable where length(word) = 15 ORDER BY frequence DESC LIMIT 20')
if len(result.collect()) == 0:
    print "le text ne contient pas de mot de quatre lettre "
else:
    max_frequence = result.collect()[0].frequence
    print(' le(s) mot(s) de quatre lettres le(s) plus fréquent')
    for element in result.collect():
        if max_frequence == element.frequence:
            print " "
            print element.word
        else:
            break

 le(s) mot(s) de quatre lettres le(s) plus fréquent
 
many-fountained
